In [ ]:
!pip install gdown inference-gpu
!pip install onnxruntime-gpu

In [ ]:
!pip install -q git+https://github.com/roboflow/sports.git

In [ ]:
!pip install ultralytics

In [ ]:
!pip install flask flask-ngrok pyngrok opencv-python numpy supervision google-generativeai werkzeug
!ngrok authtoken your_api_key

In [ ]:
import os
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

In [ ]:
#####COMBIEND CODE
import numpy as np
import supervision as sv
from inference.models.utils import get_roboflow_model
from sports.configs.soccer import SoccerPitchConfiguration
from sports.common.view import ViewTransformer

# Initialize model and class information
model = get_roboflow_model(
    model_id="cat-nifqa/1",
    api_key="your_api_key"
)

FIELD_DETECTION_MODEL = get_roboflow_model(model_id="football-field-detection-f07vi/14", api_key="your_api_key")

ModelDependencyMissing: Your `inference` configuration does not support PaliGemma model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Florence2 model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does

In [3]:
#the combiend code
class_names = model.class_names
BALL_CLASS_ID = 0  # Verify this matches your model's ball class ID

CONFIG = SoccerPitchConfiguration()

# Initialize ByteTrack with correct arguments for your supervision version
tracker = sv.ByteTrack(
    track_activation_threshold=0.2,  # Use track_activation_threshold instead of track_thresh
    lost_track_buffer=60,
    minimum_matching_threshold=0.85,
    frame_rate=25      ######30
)
##########################new edits
# tracker = sv.ByteTrack(
#     track_activation_threshold=0.3,  # زيادة العتبة لتجنب تتبع الضوضاء
#     lost_track_buffer=180,           # زيادة وقت الاحتفاظ بالتتبع (180 إطار ≈ 7 ثوانٍ)
#     minimum_matching_threshold=0.75,  # تقليل صرامة المطابقة
#     frame_rate=25
#)
# في بداية الكود (خارج الدالة callback)
persistent_ids = {}  # لتخزين المعرفات الثابتة {tracker_id: persistent_id}
current_persistent_id = 1  # لإنشاء معرفات متزايدة
########################
data_for_llm = []
# Annotators
# box_annotator = sv.BoundingBoxAnnotator()
# label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex([
        '#FF0000',  # أحمر
        '#00FF00',  # أخضر
        '#0000FF',  # أزرق
        '#FFFF00',  # أصفر
        '#FF00FF',  # أرجواني
        '#00FFFF',  # سماوي
        '#FFA500',  # برتقالي
        '#800080'   # بنفسجي غامق
    ]),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex([
        '#FF0000', '#00FF00', '#0000FF',
        '#FFFF00', '#FF00FF', '#00FFFF',
        '#FFA500', '#800080'
    ]),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),  # لون المثلثات يبقى ذهبي
    base=25,
    height=21,
    outline_thickness=1
)

vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    radius=8
)

In [ ]:
import cv2
import numpy as np
import supervision as sv
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.view import ViewTransformer

CONFIG = SoccerPitchConfiguration()

def callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
    # استدلال النموذج الرئيسي
    results = model.infer(frame)[0]
    detections = sv.Detections.from_inference(results)

    # فصل الكشف عن الكرة
    ball_mask = detections.class_id == BALL_CLASS_ID
    ball_detections = detections[ball_mask]
    other_detections = detections[~ball_mask]

    # معالجة كشف الكرة
    if len(ball_detections) > 0:
        ball_detections = ball_detections[ball_detections.confidence >= 0.1]
        padded_xyxy = sv.pad_boxes(ball_detections.xyxy, 20)
        ball_detections = sv.Detections(
            xyxy=padded_xyxy,
            confidence=ball_detections.confidence,
            class_id=ball_detections.class_id,
            data=ball_detections.data
        )

    # معالجة الكائنات الأخرى
    if len(other_detections) > 0:
        other_detections = other_detections[other_detections.confidence >= 0.8]\
            .with_nms(0.7, class_agnostic=True)

    # معالجة الكشوفات الفارغة
    ball_detections = ball_detections if hasattr(ball_detections, 'xyxy') else sv.Detections.empty()
    other_detections = other_detections if hasattr(other_detections, 'xyxy') else sv.Detections.empty()

    # دمج الكشوفات
    combined = sv.Detections.merge([ball_detections, other_detections])
    tracked_detections = tracker.update_with_detections(combined)

    # استدلال نموذج الملعب
    field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(field_result)

    # ترشيح نقاط الملعب
    confidence_mask = key_points.confidence[0] > 0.5
    filtered_xy = key_points.xy[0][confidence_mask]
    filtered_confidence = key_points.confidence[0][confidence_mask]
    frame_key_points = sv.KeyPoints(
        xy=np.expand_dims(filtered_xy, 0),
        confidence=np.expand_dims(filtered_confidence, 0)
    )

    # إنشاء محول الإحداثيات
    transformer = None
    if len(filtered_xy) >= 4:
        try:
            pitch_points = np.array(CONFIG.vertices)[confidence_mask]
            transformer = ViewTransformer(filtered_xy, pitch_points)
        except Exception as e:
            print(f"Transformation setup failed: {str(e)}")

    # جلب الإحداثيات المُسقطة
    if len(tracked_detections) > 0:
        frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        if transformer:
            try:
                pitch_anchors = transformer.transform_points(frame_anchors)
            except:
                pitch_anchors = frame_anchors  # الرجوع للإحداثيات الأصلية
        else:
            pitch_anchors = frame_anchors
    else:
        pitch_anchors = np.empty((0, 2))

    # فصل الإحداثيات حسب الفئة
    pitch_ball = []
    black_team = []
    green_team = []
    red_team = []
    white_team = []
    yellow_team = []
    referees = []
    goalkeepers = []

    for i in range(len(tracked_detections)):
        class_id = tracked_detections.class_id[i]
        if class_id == 0:  # الكرة
            pitch_ball.append(pitch_anchors[i])
        elif class_id == 1:  # black_team
            black_team.append(pitch_anchors[i])
        elif class_id == 3:  # green_team
            green_team.append(pitch_anchors[i])
        elif class_id == 4:  # red_team
            red_team.append(pitch_anchors[i])
        elif class_id == 6:  # white_team
            white_team.append(pitch_anchors[i])
        elif class_id == 7:  # yellow_team
            yellow_team.append(pitch_anchors[i])
        elif class_id == 5:  # الحكام
            referees.append(pitch_anchors[i])
        elif class_id == 2:  # حراس المرمى
            goalkeepers.append(pitch_anchors[i])

    # تحويل إلى مصفوفات
    pitch_ball = np.array(pitch_ball)
    black_team = np.array(black_team)
    green_team = np.array(green_team)
    red_team = np.array(red_team)
    white_team = np.array(white_team)
    yellow_team = np.array(yellow_team)
    referees = np.array(referees)
    goalkeepers = np.array(goalkeepers)

    # إنشاء رادار الملعب
    annotated_radar = draw_pitch(CONFIG)

    # رسم العناصر على الرادار مع تحديد thickness
    if len(pitch_ball) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=pitch_ball,
            face_color=sv.Color.WHITE,
            edge_color=sv.Color.BLACK,
            radius=10,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(black_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=black_team,
            face_color=sv.Color.from_hex('000000'),
            edge_color=sv.Color.WHITE,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(green_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=green_team,
            face_color=sv.Color.from_hex('00FF00'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(red_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=red_team,
            face_color=sv.Color.from_hex('FF0000'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(white_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=white_team,
            face_color=sv.Color.from_hex('FFFFFF'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(yellow_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=yellow_team,
            face_color=sv.Color.from_hex('FFFF00'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(referees) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=referees,
            face_color=sv.Color.from_hex('FFD700'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(goalkeepers) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=goalkeepers,
            face_color=sv.Color.from_hex('0000FF'),
            edge_color=sv.Color.BLACK,
            radius=20,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    # دمج الرادار مع الإطار الرئيسي
    if annotated_radar.size > 0:
        try:
            radar_size = (256, 256)
            resized_radar = cv2.resize(annotated_radar, radar_size)
            x = frame.shape[1] - radar_size[0] - 10
            y = 10
            frame[y:y+radar_size[1], x:x+radar_size[0]] = resized_radar
        except Exception as e:
            print(f"Radar overlay error: {str(e)}")

    # إنشاء التعليقات المرئية على الإطار الرئيسي
    labels = [f"#{tid} {class_names[cid]}"
             for cid, tid in zip(tracked_detections.class_id, tracked_detections.tracker_id)]

    annotated_frame = ellipse_annotator.annotate(frame.copy(), tracked_detections)
    annotated_frame = triangle_annotator.annotate(annotated_frame, tracked_detections)
    annotated_frame = label_annotator.annotate(annotated_frame, tracked_detections, labels)

    if len(filtered_xy) > 0:
        annotated_frame = vertex_annotator.annotate(annotated_frame, frame_key_points)

    return trace_annotator.annotate(annotated_frame, tracked_detections)

# معالجة الفيديو
sv.process_video(
    source_path="/content/0bfacc_0.mp4",
    target_path="/content/output19.mp4",
    callback=callback
)

In [ ]:
##################new edits
import cv2
import numpy as np
import supervision as sv
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.view import ViewTransformer
data_for_llm = []
CONFIG = SoccerPitchConfiguration()



# نظام المعرفات الثابتة
persistent_ids = {}
current_persistent_id = 1

def callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
    # استدلال النموذج الرئيسي
    results = model.infer(frame)[0]
    detections = sv.Detections.from_inference(results)

    # فصل الكشف عن الكرة
    ball_mask = detections.class_id == BALL_CLASS_ID
    ball_detections = detections[ball_mask]
    other_detections = detections[~ball_mask]

    # معالجة كشف الكرة
    if len(ball_detections) > 0:
        ball_detections = ball_detections[ball_detections.confidence >= 0.1]
        padded_xyxy = sv.pad_boxes(ball_detections.xyxy, 20)
        ball_detections = sv.Detections(
            xyxy=padded_xyxy,
            confidence=ball_detections.confidence,
            class_id=ball_detections.class_id,
            data=ball_detections.data
        )

    # معالجة الكائنات الأخرى
    if len(other_detections) > 0:
        other_detections = other_detections[other_detections.confidence >= 0.8]\
            .with_nms(0.7, class_agnostic=True)

    # معالجة الكشوفات الفارغة
    ball_detections = ball_detections if hasattr(ball_detections, 'xyxy') else sv.Detections.empty()
    other_detections = other_detections if hasattr(other_detections, 'xyxy') else sv.Detections.empty()

    # دمج الكشوفات
    combined = sv.Detections.merge([ball_detections, other_detections])
    tracked_detections = tracker.update_with_detections(combined)

    # إنشاء معرفات ثابتة للاعبين الجدد
    global current_persistent_id
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        if tracker_id not in persistent_ids:
            persistent_ids[tracker_id] = current_persistent_id
            current_persistent_id += 1

    # استدلال نموذج الملعب
    field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(field_result)

    # ترشيح نقاط الملعب
    confidence_mask = key_points.confidence[0] > 0.5
    filtered_xy = key_points.xy[0][confidence_mask]
    filtered_confidence = key_points.confidence[0][confidence_mask]
    frame_key_points = sv.KeyPoints(
        xy=np.expand_dims(filtered_xy, 0),
        confidence=np.expand_dims(filtered_confidence, 0)
    )

    # إنشاء محول الإحداثيات
    transformer = None
    if len(filtered_xy) >= 4:
        try:
            pitch_points = np.array(CONFIG.vertices)[confidence_mask]
            transformer = ViewTransformer(filtered_xy, pitch_points)
        except Exception as e:
            print(f"Transformation setup failed: {str(e)}")

    # جلب الإحداثيات المُسقطة
    if len(tracked_detections) > 0:
        frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        if transformer:
            try:
                pitch_anchors = transformer.transform_points(frame_anchors)
            except:
                pitch_anchors = frame_anchors  # الرجوع للإحداثيات الأصلية
        else:
            pitch_anchors = frame_anchors
    else:
        pitch_anchors = np.empty((0, 2))

    current_frame_data = {
        "frame_id": frame_id,
        "timestamp": round(frame_id / 25, 2),
        "objects": [],
        "keypoints": []
    }

    # إضافة الكائنات مع الإحداثيات المُسقطة والمعرفات الثابتة
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_id = tracked_detections.class_id[i]
        obj = {
            "persistent_id": persistent_ids[tracker_id],  # إضافة المعرف الثابت
            "tracker_id": tracker_id,
            "class": class_names[class_id],
            "confidence": float(tracked_detections.confidence[i]),
            "position": pitch_anchors[i].tolist() if len(pitch_anchors) > i else []
        }
        current_frame_data["objects"].append(obj)

    # إضافة نقاط الملعب
    for (x, y), conf in zip(filtered_xy, filtered_confidence):
        current_frame_data["keypoints"].append({
            "position": [round(x, 1), round(y, 1)],
            "confidence": round(conf, 2)
        })

    # تحديث بيانات الإطار الكامل مع كرة
    has_ball = any(obj['class'] == class_names[BALL_CLASS_ID] for obj in current_frame_data["objects"])
    if has_ball:
        global last_ball_frame_data
        last_ball_frame_data = current_frame_data.copy()

    # تخزين البيانات كل 25 إطارًا مع التعويض
    if frame_id % 25 == 0:
        if has_ball:
            data_for_llm.append(current_frame_data)
        else:
            if last_ball_frame_data is not None:
                data_for_llm.append(last_ball_frame_data)
    # فصل الإحداثيات حسب الفئة
    pitch_ball = []
    black_team = []
    green_team = []
    red_team = []
    white_team = []
    yellow_team = []
    referees = []
    goalkeepers = []

    for i in range(len(tracked_detections)):
        class_id = tracked_detections.class_id[i]
        if class_id == 0:  # الكرة
            pitch_ball.append(pitch_anchors[i])
        elif class_id == 1:  # black_team
            black_team.append(pitch_anchors[i])
        elif class_id == 3:  # green_team
            green_team.append(pitch_anchors[i])
        elif class_id == 4:  # red_team
            red_team.append(pitch_anchors[i])
        elif class_id == 6:  # white_team
            white_team.append(pitch_anchors[i])
        elif class_id == 7:  # yellow_team
            yellow_team.append(pitch_anchors[i])
        elif class_id == 5:  # الحكام
            referees.append(pitch_anchors[i])
        elif class_id == 2:  # حراس المرمى
            goalkeepers.append(pitch_anchors[i])

    # تحويل إلى مصفوفات
    pitch_ball = np.array(pitch_ball)
    black_team = np.array(black_team)
    green_team = np.array(green_team)
    red_team = np.array(red_team)
    white_team = np.array(white_team)
    yellow_team = np.array(yellow_team)
    referees = np.array(referees)
    goalkeepers = np.array(goalkeepers)

    # إنشاء رادار الملعب
    annotated_radar = draw_pitch(CONFIG)

    # رسم العناصر على الرادار
    if len(pitch_ball) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=pitch_ball,
            face_color=sv.Color.WHITE,
            edge_color=sv.Color.BLACK,
            radius=10,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(black_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=black_team,
            face_color=sv.Color.from_hex('000000'),
            edge_color=sv.Color.WHITE,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(green_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=green_team,
            face_color=sv.Color.from_hex('00FF00'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(red_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=red_team,
            face_color=sv.Color.from_hex('FF0000'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(white_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=white_team,
            face_color=sv.Color.from_hex('FFFFFF'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(yellow_team) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=yellow_team,
            face_color=sv.Color.from_hex('FFFF00'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(referees) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=referees,
            face_color=sv.Color.from_hex('FFD700'),
            edge_color=sv.Color.BLACK,
            radius=16,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    if len(goalkeepers) > 0:
        annotated_radar = draw_points_on_pitch(
            config=CONFIG,
            xy=goalkeepers,
            face_color=sv.Color.from_hex('0000FF'),
            edge_color=sv.Color.BLACK,
            radius=20,
            thickness=2,  # إضافة thickness
            pitch=annotated_radar
        )

    # ... (كرر الأقسام السابقة لـ green_team, red_team, إلخ)

    # دمج الرادار مع الإطار الرئيسي
    if annotated_radar.size > 0:
        try:
            radar_size = (256, 256)
            resized_radar = cv2.resize(annotated_radar, radar_size)
            x = frame.shape[1] - radar_size[0] - 10
            y = 10
            frame[y:y+radar_size[1], x:x+radar_size[0]] = resized_radar
        except Exception as e:
            print(f"Radar overlay error: {str(e)}")

    # إنشاء التعليقات المرئية مع المعرفات الثابتة
    labels = []
    for i in range(len(tracked_detections)):
        cid = tracked_detections.class_id[i]
        tid = tracked_detections.tracker_id[i]
        labels.append(f"#{persistent_ids[tid]} {class_names[cid]}")

    annotated_frame = ellipse_annotator.annotate(frame.copy(), tracked_detections)
    annotated_frame = triangle_annotator.annotate(annotated_frame, tracked_detections)
    annotated_frame = label_annotator.annotate(annotated_frame, tracked_detections, labels)

    if len(filtered_xy) > 0:
        annotated_frame = vertex_annotator.annotate(annotated_frame, frame_key_points)

    return trace_annotator.annotate(annotated_frame, tracked_detections)

sv.process_video(
    source_path="/content/2e57b9_0.mp4",
    target_path="/content/output23.mp4",
    callback=callback
)

In [ ]:
import cv2
import numpy as np
import random
import supervision as sv
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.view import ViewTransformer

# Global variables for team tracking
HOME_TEAM = None
AWAY_TEAM = None
data_for_llm = []
CONFIG = SoccerPitchConfiguration()
BALL_CLASS_ID = 0  # Assuming ball is class 0


# Persistent ID system
persistent_ids = {}
current_persistent_id = 1

def classify_tactical_zone(x, y, is_home_team):
    pitch_width = 12000.0

    # تقسيم الملعب إلى ثلثين أفقيًا
    defensive_third = pitch_width * 0.33  # 3960
    attacking_third = pitch_width * 0.66  # 7920

    if is_home_team:
        # الفريق المنزل يهاجم على اليسار (x صغير)
        if x < defensive_third:
            return "Attacking Third"
        elif x > attacking_third:
            return "Defensive Third"
        else:
            return "Midfield Third"
    else:
        # الفريق الخصم يهاجم على اليمين (x كبير)
        if x > attacking_third:
            return "Attacking Third"
        elif x < defensive_third:
            return "Defensive Third"
        else:
            return "Midfield Third"

def callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
    global HOME_TEAM, AWAY_TEAM, current_persistent_id

    # Main model inference
    results = model.infer(frame)[0]
    detections = sv.Detections.from_inference(results)

    # Separate ball from other detections
    ball_mask = detections.class_id == BALL_CLASS_ID
    ball_detections = detections[ball_mask]
    other_detections = detections[~ball_mask]

    # Process detections
    if len(ball_detections) > 0:
        ball_detections = ball_detections[ball_detections.confidence >= 0.1]
        ball_detections = sv.Detections(
            xyxy=sv.pad_boxes(ball_detections.xyxy, 20),
            confidence=ball_detections.confidence,
            class_id=ball_detections.class_id,
            data=ball_detections.data
        )
    if len(other_detections) > 0:
        other_detections = other_detections[other_detections.confidence >= 0.8]\
            .with_nms(0.7, class_agnostic=True)

    # Merge detections
    combined = sv.Detections.merge([ball_detections, other_detections])
    tracked_detections = tracker.update_with_detections(combined)

    # Create persistent IDs
    for tracker_id in tracked_detections.tracker_id:
        if tracker_id not in persistent_ids:
            persistent_ids[tracker_id] = current_persistent_id
            current_persistent_id += 1

    # استدلال نموذج الملعب
    field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(field_result)

    # ترشيح نقاط الملعب
    confidence_mask = key_points.confidence[0] > 0.5
    filtered_xy = key_points.xy[0][confidence_mask]
    filtered_confidence = key_points.confidence[0][confidence_mask]
    frame_key_points = sv.KeyPoints(
        xy=np.expand_dims(filtered_xy, 0),
        confidence=np.expand_dims(filtered_confidence, 0)
    )

    # إنشاء محول الإحداثيات
    transformer = None
    if len(filtered_xy) >= 4:
        try:
            pitch_points = np.array(CONFIG.vertices)[confidence_mask]
            transformer = ViewTransformer(filtered_xy, pitch_points)
        except Exception as e:
            print(f"Transformation setup failed: {str(e)}")

    # Get projected coordinates
    if len(tracked_detections) > 0:
        frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        if transformer:
            try:
                pitch_anchors = transformer.transform_points(frame_anchors)
            except:
                pitch_anchors = frame_anchors
        else:
            pitch_anchors = frame_anchors
    else:
        pitch_anchors = np.empty((0, 2))

    # Create frame data
    current_frame_data = {
        "frame_id": frame_id,
        "timestamp": round(frame_id / 25, 2),
        "objects": []
    }

    # Detect teams automatically
    detected_teams = set()
    for i in range(len(tracked_detections)):
        class_id = tracked_detections.class_id[i]
        if class_id in [1, 2, 3, 4, 5, 6, 7]:  # Exclude referees (class 5)
            team_name = class_names[class_id]
            detected_teams.add(team_name)

    # Assign home/away teams
    if not HOME_TEAM and len(detected_teams) >= 2:
        HOME_TEAM, AWAY_TEAM = random.sample(list(detected_teams), 2)
    elif len(detected_teams) == 1:
        HOME_TEAM = list(detected_teams)[0]

    # Process each object and calculate zones
    tracker_id_to_zone = {}
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_id = tracked_detections.class_id[i]
        class_name = class_names[class_id]
        position = pitch_anchors[i].tolist() if len(pitch_anchors) > i else []

        # Calculate zone for team players
        if class_name in ["black_team", "green_team", "red_team", "white_team", "yellow_team"]:
            is_home = (class_name == HOME_TEAM) if HOME_TEAM else False
            x, y = position if len(position) == 2 else (0, 0)
            zone = classify_tactical_zone(x, y, is_home)
            tracker_id_to_zone[tracker_id] = zone
            print(f"x: {x}, is_home: {is_home}, zone: {zone}")
        else:
            tracker_id_to_zone[tracker_id] = "Unknown"

        obj = {
            "persistent_id": persistent_ids[tracker_id],
            "tracker_id": tracker_id,
            "class": class_name,
            "confidence": float(tracked_detections.confidence[i]),
            "position": position,
            "zone": tracker_id_to_zone[tracker_id]
        }
        current_frame_data["objects"].append(obj)

    # Store data every 25 frames
    if frame_id % 25 == 0:
        data_for_llm.append(current_frame_data)

    # Draw radar elements
    annotated_radar = draw_pitch(CONFIG)
    team_colors = {
        "white_team": sv.Color.WHITE,
        "red_team": sv.Color.RED,
        "green_team": sv.Color.GREEN,
        "yellow_team": sv.Color.YELLOW,
        "black_team": sv.Color.BLACK
    }

    for team in team_colors:
        team_detections = [pitch_anchors[i] for i in range(len(tracked_detections))
                         if class_names[tracked_detections.class_id[i]] == team]
        if team_detections:
            annotated_radar = draw_points_on_pitch(
                config=CONFIG,
                xy=np.array(team_detections),
                face_color=team_colors[team],
                edge_color=sv.Color.BLACK,
                radius=16,
                thickness=2,
                pitch=annotated_radar
            )

    # Overlay radar on main frame
    if annotated_radar.size > 0:
        try:
            radar_size = (256, 256)
            resized_radar = cv2.resize(annotated_radar, radar_size)
            frame[10:10+radar_size[1], -radar_size[0]-10:-10] = resized_radar
        except Exception as e:
            print(f"Radar overlay error: {str(e)}")

    # Create visual annotations with zones instead of labels
    labels = []
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_name = class_names[tracked_detections.class_id[i]]

        # Show zone for team players, hide labels for others
        if class_name in team_colors:
            labels.append(tracker_id_to_zone.get(tracker_id, "Unknown"))
        else:
            labels.append("")

    # Apply annotations
    annotated_frame = ellipse_annotator.annotate(frame.copy(), tracked_detections)
    annotated_frame = triangle_annotator.annotate(annotated_frame, tracked_detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame,
        tracked_detections,
        labels  # Now shows tactical zones instead of IDs
    )

    return trace_annotator.annotate(annotated_frame, tracked_detections)

# Process video
sv.process_video(
    source_path="/content/2e57b9_0.mp4",
    target_path="/content/output34.mp4",
    callback=callback
)

In [ ]:
import cv2
import numpy as np
import random
import supervision as sv
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.view import ViewTransformer

# Global variables for team tracking
HOME_TEAM = None
AWAY_TEAM = None
data_for_llm = []
CONFIG = SoccerPitchConfiguration()
BALL_CLASS_ID = 0  # Assuming ball is class 0

SOURCE_VIDEO_PATH = "/content/2e57b9_0.mp4"  # أدخل مسار الفيديو المصدر
TARGET_VIDEO_PATH = "/content/output_full_zones.mp4" # أدخل مسار الفيديو الهدف

# Persistent ID system
persistent_ids = {}
current_persistent_id = 1
# --- تحديد الفرق الثابت (مهم!) ---
# قم بتعيين اسم الفئة للفريق الذي يهاجم المرمى الأيسر (قيم X الصغيرة)
KNOWN_HOME_TEAM = "white_team"
# قم بتعيين اسم الفئة للفريق الذي يهاجم المرمى الأيمن (قيم X الكبيرة)
KNOWN_AWAY_TEAM = "black_team"


def classify_tactical_zone(x: float, y: float, is_home_team: bool, pitch_config: SoccerPitchConfiguration) -> str:
    """
    Classifies the tactical zone of a player based on their x-coordinate
    relative to their team's attacking direction.
    """
    try:
        # الحصول على أبعاد الملعب
        pitch_width = pitch_config.width
        pitch_height = pitch_config.height
    except AttributeError:
        print("Warning: Could not get pitch dimensions from pitch_config. Using default 10000.")
        pitch_width = 10000.0  # قيمة افتراضية
        pitch_height = 6000.0  # قيمة افتراضية

    if pitch_width <= 0:
        return "Unknown Zone (Invalid Pitch Width)"

    # تقسيم الملعب إلى ثلثين أفقيًا
    first_third_line = pitch_width * (1 / 3)
    second_third_line = pitch_width * (2 / 3)

    zone = "Unknown Zone"  # Default value

    if is_home_team:
        # الفريق الأساسي (Home) يهاجم نحو اليسار (قيم X الصغيرة)
        if x < first_third_line:
            zone = "Attacking Third"
        elif x > second_third_line:
            zone = "Defensive Third"
        else:
            zone = "Midfield Third"
    else:
        # الفريق الضيف (Away) يهاجم نحو اليمين (قيم X الكبيرة)
        if x > second_third_line:
            zone = "Attacking Third"
        elif x < first_third_line:
            zone = "Defensive Third"
        else:
            zone = "Midfield Third"
    # --- (اختياري) إضافة تصنيف عمودي (أجنحة/مركز) ---
    # if zone != "Unknown Zone" and pitch_height > 0:
    #     center_third_y_start = pitch_height * (1/3)
    #     center_third_y_end = pitch_height * (2/3)
    #     # قد تحتاج لعكس المنطق إذا كان y=0 هو الأسفل
    #     if y < center_third_y_start:
    #         zone += " (Top Flank)" # جناح علوي
    #     elif y > center_third_y_end:
    #         zone += " (Bot Flank)" # جناح سفلي
    #     else:
    #         zone += " (Center)" # مركز
    # --------------------------------------------------

    return zone

def callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
    global HOME_TEAM, AWAY_TEAM, current_persistent_id, data_for_llm, persistent_ids

    # 1. استدلال النموذج الرئيسي
    try:
        results = model.infer(frame)[0]
        detections = sv.Detections.from_inference(results)
    except Exception as e:
        print(f"Model inference error: {e}")
        return frame

    # 2. فصل الكرة ومعالجة الاكتشافات
    ball_mask = detections.class_id == BALL_CLASS_ID
    ball_detections = detections[ball_mask]
    other_detections = detections[~ball_mask]

    # ترشيح حسب الثقة
    if len(ball_detections) > 0:
        ball_detections = ball_detections[ball_detections.confidence >= 0.1]
        ball_detections = sv.Detections(
            xyxy=sv.pad_boxes(ball_detections.xyxy, 20),
            confidence=ball_detections.confidence,
            class_id=ball_detections.class_id,
            data=ball_detections.data
        )
    if len(other_detections) > 0:
        other_detections = other_detections[other_detections.confidence >= 0.6]\
            .with_nms(0.7, class_agnostic=True)

    # 3. دمج الاكتشافات وتطبيق التتبع
    combined = sv.Detections.merge([ball_detections, other_detections])
    tracked_detections = tracker.update_with_detections(combined)

    # 4. إنشاء معرفات ثابتة
    for tracker_id in tracked_detections.tracker_id:
        if tracker_id not in persistent_ids:
            persistent_ids[tracker_id] = current_persistent_id
            current_persistent_id += 1

    # 5. استدلال نموذج الملعب وتحويل الإحداثيات
    transformer = None
    try:
        field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        key_points = sv.KeyPoints.from_inference(field_result)
        if len(key_points.xy) > 0 and len(key_points.xy[0]) > 0:
            confidence_mask = key_points.confidence[0] > 0.5
            filtered_xy = key_points.xy[0][confidence_mask]
            if len(filtered_xy) >= 4:
                pitch_points = np.array(CONFIG.vertices)[confidence_mask]
                transformer = ViewTransformer(filtered_xy, pitch_points)
    except Exception as e:
        print(f"Transformation error: {e}")

    # 6. الحصول على الإحداثيات المسقطة
    if len(tracked_detections) > 0:
        frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        if transformer:
            try:
                pitch_anchors = transformer.transform_points(frame_anchors)
            except:
                pitch_anchors = frame_anchors
        else:
            pitch_anchors = frame_anchors
    else:
        pitch_anchors = np.empty((0, 2))

    # 7. تحديد الفرق تلقائيًا
    detected_teams = set()
    for i in range(len(tracked_detections)):
        class_id = tracked_detections.class_id[i]
        if class_id in [1, 2, 3, 4, 5, 6, 7]:
            team_name = class_names[class_id]
            detected_teams.add(team_name)

    if not HOME_TEAM and KNOWN_HOME_TEAM in detected_teams:
        HOME_TEAM = KNOWN_HOME_TEAM
    if not AWAY_TEAM and KNOWN_AWAY_TEAM in detected_teams:
        AWAY_TEAM = KNOWN_AWAY_TEAM

    # 8. تصنيف المناطق التكتيكية
    tracker_id_to_zone = {}
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_id = tracked_detections.class_id[i]
        class_name = class_names[class_id]
        position = pitch_anchors[i].tolist() if len(pitch_anchors) > i else []

        if class_name in TEAM_CLASS_NAMES and len(position) == 2:
            is_home = (class_name == HOME_TEAM) if HOME_TEAM else False
            x, y = position
            zone = classify_tactical_zone(x, y, is_home, CONFIG)
            tracker_id_to_zone[tracker_id] = zone
        else:
            tracker_id_to_zone[tracker_id] = "Unknown"

    # 9. رسم المناطق على الإطار
    labels = []
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_name = class_names[tracked_detections.class_id[i]]
        zone = tracker_id_to_zone.get(tracker_id, "Unknown")

        if class_name in TEAM_CLASS_NAMES:
            labels.append(zone.split()[0][:4])  # اختصار مثل Att, Mid, Def
        else:
            labels.append("")

    # 10. تطبيق التعليقات التوضيحية
    annotated_frame = ellipse_annotator.annotate(frame.copy(), tracked_detections)
    annotated_frame = triangle_annotator.annotate(annotated_frame, tracked_detections)
    annotated_frame = label_annotator.annotate(annotated_frame, tracked_detections, labels)

    return trace_annotator.annotate(annotated_frame, tracked_detections)
# --- بدء معالجة الفيديو ---
print("Starting video processing...")
try:
    sv.process_video(
        source_path=SOURCE_VIDEO_PATH,
        target_path=TARGET_VIDEO_PATH,
        callback=callback
    )
    print(f"Video processing complete. Output saved to: {TARGET_VIDEO_PATH}")
    # طباعة بيانات LLM المجمعة إذا تم تجميعها
    # import json
    # with open("output_data_llm.json", "w") as f:
    #    json.dump(data_for_llm, f, indent=4)
    # print("LLM data saved to output_data_llm.json")

except FileNotFoundError:
    print(f"ERROR: Source video file not found at '{SOURCE_VIDEO_PATH}'")
except Exception as e:
    print(f"An unexpected error occurred during video processing: {e}")

In [ ]:
import cv2
import numpy as np
import random
import supervision as sv
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.view import ViewTransformer

# Global variables for team tracking
HOME_TEAM = None
AWAY_TEAM = None
KNOWN_HOME_TEAM = "white_team"  
KNOWN_AWAY_TEAM = "black_team"   
data_for_llm = []
CONFIG = SoccerPitchConfiguration()
BALL_CLASS_ID = 0  # Assuming ball is class 0
# Persistent ID system
persistent_ids = {}
current_persistent_id = 1

def classify_tactical_zone(x: float, y: float, is_home_team: bool, pitch_config: SoccerPitchConfiguration) -> str:
    pitch_width = 12000.0
    pitch_height = 8000.0



    first_third_line = pitch_width * (1 / 3)
    second_third_line = pitch_width * (2 / 3)
    zone = "Unknown Zone"

    if is_home_team:
        if x < first_third_line:
            zone = "Attacking Third"
        elif x > second_third_line:
            zone = "Defensive Third"
        else:
            zone = "Midfield Third"
    else:
        if x > second_third_line:
            zone = "Attacking Third"
        elif x < first_third_line:
            zone = "Defensive Third"
        else:
            zone = "Midfield Third"
    
    if zone != "Unknown Zone":
        center_third_y_start = pitch_height * (1/3)
        center_third_y_end = pitch_height * (2/3)
        
        if y < center_third_y_start:
            zone += " (Top Flank)"  # جناح علوي
        elif y > center_third_y_end:
            zone += " (Bot Flank)"  # جناح سفلي
        else:
            zone += " (Center)"     # المنطقة المركزية

    return zone

def callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
    global HOME_TEAM, AWAY_TEAM, current_persistent_id
    # Main model inference
    results = model.infer(frame)[0]
    detections = sv.Detections.from_inference(results)

    # Separate ball from other detections
    ball_mask = detections.class_id == BALL_CLASS_ID
    ball_detections = detections[ball_mask]
    other_detections = detections[~ball_mask]

    # Process detections
    if len(ball_detections) > 0:
        ball_detections = ball_detections[ball_detections.confidence >= 0.1]
        ball_detections = sv.Detections(
            xyxy=sv.pad_boxes(ball_detections.xyxy, 20),
            confidence=ball_detections.confidence,
            class_id=ball_detections.class_id,
            data=ball_detections.data
        )
    if len(other_detections) > 0:
        other_detections = other_detections[other_detections.confidence >= 0.8]\
            .with_nms(0.7, class_agnostic=True)

    # Merge detections
    combined = sv.Detections.merge([ball_detections, other_detections])
    tracked_detections = tracker.update_with_detections(combined)

    # Create persistent IDs
    for tracker_id in tracked_detections.tracker_id:
        if tracker_id not in persistent_ids:
            persistent_ids[tracker_id] = current_persistent_id
            current_persistent_id += 1

    # استدلال نموذج الملعب
    field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(field_result)

    # ترشيح نقاط الملعب
    confidence_mask = key_points.confidence[0] > 0.5
    filtered_xy = key_points.xy[0][confidence_mask]
    filtered_confidence = key_points.confidence[0][confidence_mask]
    frame_key_points = sv.KeyPoints(
        xy=np.expand_dims(filtered_xy, 0),
        confidence=np.expand_dims(filtered_confidence, 0)
    )

    # إنشاء محول الإحداثيات
    transformer = None
    if len(filtered_xy) >= 4:
        try:
            pitch_points = np.array(CONFIG.vertices)[confidence_mask]
            transformer = ViewTransformer(filtered_xy, pitch_points)
        except Exception as e:
            print(f"Transformation setup failed: {str(e)}")

    # Get projected coordinates
    if len(tracked_detections) > 0:
        frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        if transformer:
            try:
                pitch_anchors = transformer.transform_points(frame_anchors)
            except:
                pitch_anchors = frame_anchors
        else:
            pitch_anchors = frame_anchors
    else:
        pitch_anchors = np.empty((0, 2))

    # Create frame data
    current_frame_data = {
        "frame_id": frame_id,
        "timestamp": round(frame_id / 25, 2),
        "objects": []
    }

    # Detect teams automatically
    detected_teams = set()
    for i in range(len(tracked_detections)):
        class_id = tracked_detections.class_id[i]
        if class_id in [1, 3, 4, 6, 7]:  # Exclude referees (class 5)
            team_name = class_names[class_id]
            detected_teams.add(team_name)

    # Assign home/away teams (الكود الجديد)
    if not HOME_TEAM and KNOWN_HOME_TEAM in detected_teams:
        HOME_TEAM = KNOWN_HOME_TEAM
    if not AWAY_TEAM and KNOWN_AWAY_TEAM in detected_teams:
        AWAY_TEAM = KNOWN_AWAY_TEAM

    # Process each object and calculate zones
    tracker_id_to_zone = {}
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_id = tracked_detections.class_id[i]
        class_name = class_names[class_id]
        position = pitch_anchors[i].tolist() if len(pitch_anchors) > i else []

        # Calculate zone for team players
        if class_name in ["black_team", "green_team", "red_team", "white_team", "yellow_team"]:
            is_home = (class_name == HOME_TEAM) if HOME_TEAM else False
            x, y = position if len(position) == 2 else (0, 0)
            zone = classify_tactical_zone(x, y, is_home,pitch_config=CONFIG)
            tracker_id_to_zone[tracker_id] = zone
            # print(f"x: {x}, is_home: {is_home}, zone: {zone}")
        else:
            tracker_id_to_zone[tracker_id] = "Unknown"

        # obj = {
        #     "persistent_id": persistent_ids[tracker_id],
        #     "tracker_id": tracker_id,
        #     "class": class_name,
        #     "confidence": float(tracked_detections.confidence[i]),
        #     "position": position,
        #     "zone": tracker_id_to_zone[tracker_id]
        # }
        # current_frame_data["objects"].append(obj)
        tracker_id_int = int(tracker_id) 
        # تأكد من أن persistent_id هو نوع int قياسي (احتياطي)
        persistent_id_int = int(persistent_ids[tracker_id])

        obj = {
            "persistent_id": persistent_id_int,
            "tracker_id": tracker_id_int, # <--- استخدام القيمة المحولة
            "class": class_name,
            "confidence": float(tracked_detections.confidence[i]), # يبقى كما هو
            "position": position, # .tolist() يجب أن يكون كافيًا هنا
            "zone": tracker_id_to_zone[tracker_id] # المفتاح هنا لا يزال tracker_id الأصلي، هذا جيد
        }
        current_frame_data["objects"].append(obj)

    # Store data every 25 frames
    if frame_id % 25 == 0:
        data_for_llm.append(current_frame_data)

    # Draw radar elements
    annotated_radar = draw_pitch(CONFIG)
    team_colors = {
        "white_team": sv.Color.WHITE,
        "red_team": sv.Color.RED,
        "green_team": sv.Color.GREEN,
        "yellow_team": sv.Color.YELLOW,
        "black_team": sv.Color.BLACK
    }

    for team in team_colors:
        team_detections = [pitch_anchors[i] for i in range(len(tracked_detections))
                         if class_names[tracked_detections.class_id[i]] == team]
        if team_detections:
            annotated_radar = draw_points_on_pitch(
                config=CONFIG,
                xy=np.array(team_detections),
                face_color=team_colors[team],
                edge_color=sv.Color.BLACK,
                radius=16,
                thickness=2,
                pitch=annotated_radar
            )

    # Overlay radar on main frame
    if annotated_radar.size > 0:
        try:
            radar_size = (256, 256)
            resized_radar = cv2.resize(annotated_radar, radar_size)
            frame[10:10+radar_size[1], -radar_size[0]-10:-10] = resized_radar
        except Exception as e:
            print(f"Radar overlay error: {str(e)}")

    # Create visual annotations with zones instead of labels
    labels = []
    for i in range(len(tracked_detections)):
        tracker_id = tracked_detections.tracker_id[i]
        class_name = class_names[tracked_detections.class_id[i]]

        # Show zone for team players, hide labels for others
        if class_name in team_colors:
            labels.append(tracker_id_to_zone.get(tracker_id, "Unknown"))
        else:
            labels.append("")

    # Apply annotations
    annotated_frame = ellipse_annotator.annotate(frame.copy(), tracked_detections)
    annotated_frame = triangle_annotator.annotate(annotated_frame, tracked_detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame,
        tracked_detections,
        labels  # Now shows tactical zones instead of IDs
    )

    return trace_annotator.annotate(annotated_frame, tracked_detections)

# Process video
sv.process_video(
    source_path="/kaggle/working/2e57b9_0.mp4",
    target_path="/kaggle/working/output40.mp4",
    callback=callback
)

In [ ]:
# pip install -q -U google-generativeai

import google.generativeai as genai
import os
import json


try:
    
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        
        GOOGLE_API_KEY = "YOUR_API_KEY" # 
        if GOOGLE_API_KEY == "YOUR_API_KEY":
             print("Please replace 'YOUR_API_KEY' with your actual Google API key.")
             exit() 

    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API Key configured.")

except Exception as e:
    print(f"Error configuring Gemini API key: {e}")
    print("Please ensure the GOOGLE_API_KEY environment variable is set or provide the key directly (for testing).")
    exit()


def analyze_with_gemini(data_list, analysis_request):
    """
    ترسل البيانات المجمعة إلى Gemini API وتطلب تحليلًا محددًا.

    Args:
        data_list (list): قائمة القواميس التي تحتوي على بيانات الإطارات.
        analysis_request (str): النص الذي يصف التحليل المطلوب من Gemini.

    Returns:
        str: استجابة التحليل من Gemini، أو رسالة خطأ.
    """
    if not data_list:
        return "لا توجد بيانات لتحليلها."

    
    try:
        data_json = json.dumps(data_list, indent=2, ensure_ascii=False)
    except Exception as e:
        return f"خطأ في تحويل البيانات إلى JSON: {e}"

    
    model = genai.GenerativeModel('gemini-2.5-pro-exp-03-25')#gemini-1.5-pro-latest

    # صياغة الطلب (Prompt)
    prompt = f"""
    تحليل بيانات مباراة كرة قدم:

    البيانات التالية تم استخراجها من مقطع فيديو لمباراة كرة قدم. كل عنصر في القائمة يمثل بيانات إطار معين تم التقاطه كل ثانية (تقريبًا).
    كل إطار يحتوي على قائمة بالكائنات المكتشفة (لاعبين، الكرة) مع معرفاتها، نوعها (الفريق أو الكرة)، موقعها التقريبي في الملعب (الإحداثيات المحولة)، والمنطقة التكتيكية التي يتواجد فيها اللاعب.

    البيانات:
    ```json
    {data_json}
    ```

    المطلوب:
    {analysis_request}

    يرجى تقديم التحليل باللغة العربية.
    """

    print("\n--- إرسال الطلب إلى Gemini ---")
    # print(f"Prompt (first 500 chars): {prompt[:500]}...") # لعرض جزء من الطلب للتأكد

    try:
        
        response = model.generate_content(prompt)
        print("--- تم استلام الرد من Gemini ---")
        return response.text
    except Exception as e:
        # التعامل مع الأخطاء المحتملة (مثل تجاوز حدود الإدخال، مشاكل الشبكة)
        print(f"حدث خطأ أثناء استدعاء Gemini API: {e}")
        # محاولة فهم الخطأ (قد يكون بسبب حجم البيانات)
        if "quota" in str(e).lower():
             return f"خطأ: تجاوز الحصة المخصصة لواجهة برمجة التطبيقات. حاول مرة أخرى لاحقًا."
        if "size" in str(e).lower() or "token" in str(e).lower():
            return f"خطأ: حجم البيانات المرسلة كبير جدًا وقد يتجاوز حدود Gemini. حاول تقليل عدد الإطارات المرسلة أو تبسيط البيانات."
        return f"خطأ غير متوقع من Gemini API: {e}"


if data_for_llm:
    # مثال على طلب تحليل:
    analysis_prompt = """
    الهدف:
تحليل بيانات الرؤية الحاسوبية لمباراة كرة قدم لاستخراج نقاط القوة ، نقاط الضعف ، التكتيكات الفعالة ، والمجالات التي تحتاج تحسينًا ، مع تقديم توصيات عملية للمدرب.

التعليمات :
تحليل حركة اللاعبين والكرة:
تتبع مسارات اللاعبين والكرة عبر الإطارات لتحديد:
نقاط القوة : لاعبون ذوي حركة سريعة أو دقة في التمركز (مثال: white_team #7 في الهجوم).
نقاط الضعف : لاعبون بطيئون أو خارج التغطية الدفاعية (مثال: black_team #24 في المناطق الدفاعية).
اكتشاف أنماط التسارع/التباطؤ التي تشير إلى إجهاد أو انضباط تكتيكي.
تحليل السيطرة على المناطق:
حدد المناطق التي يُسيطر عليها كل فريق (مثل ثلث الهجوم أو الدفاع).
ابحث عن ثغرات :
هل يوجد تركز مفرط للاعبي الفريق الأبيض في الوسط، مما يعرض الدفاع للخطر؟
هل الفريق الأسود يستغل الأطراف بشكل كافٍ؟
اكتشاف الأحداث الحرجة:
الاستحواذ : لاعبون قريبون من الكرة مع Conf ≥ 0.85 (مثال: black_team #19 في منتصف الملعب).
التمريرات الناجحة/الفاشلة : تمريرات طويلة أو قصيرة تشير إلى تنظيم هجومي أو فوضى.
التسديدات : كرات نحو المرمى مع ملاحظة مدى قرب المدافع (مثال: تسديدة white_team #9 في الإطار 600).
التدخلات الدفاعية : اعتراضات أو تدخلات فاشلة تكشف عن ضعف في التغطية.
تقييم الاستراتيجيات:
نقاط القوة التكتيكية :
هل يستخدم الفريق الأبيض تشكيلًا دفاعيًا فعالًا (مثل 4-4-2)؟
هل الفريق الأسود يشن هجمات مرتدة سريعة؟
نقاط الضعف التكتيكية :
هل هناك فجوات بين خطوط الفريق (مثل مسافة كبيرة بين الدفاع والوسط)؟
هل الحراس أو المدافعون غير قادرين على التعامل مع الكرات العرضية؟
تقييم الأداء الفردي:
لاعبون بـ تفاعل عالٍ : مشاركة في أغلب الأحداث (مثال: black_team #19 في الوسط).
لاعبون بـ أداء ضعيف : أخطاء متكررة أو مواقع غير مناسبة (مثال: red_team #23 مع Conf < 0.80).
اقترح مراجعة الفيديو للتأكد من الأحداث الحرجة (مثل التسديدات أو التدخلات).
متطلبات الإخراج المُعدلة:
ملخص تنفيذي للمدرب:
نقاط القوة الرئيسية لكل فريق (مثال: "سيطرة black_team على الوسط").
نقاط الضعف الحرجة (مثال: "ضعف تغطية white_team للأطراف").
توصيات فورية (مثال: "تدريب black_team #24 على السرعة في المناطق الدفاعية").
وصف بصري تكتيكي:
مخططات نصية لتمركز اللاعبين في أحداث حرجة (مثال: "في الإطار 600، white_team #7 في موقع تسديد مثالي").

مثال تحليل للمدرب:

"في الإطار 600، يظهر black_team #19 سيطرة واضحة على الوسط، لكنه يترك مساحة كبيرة في الجناح الأيمن. يُوصى بتحريك black_team #12 لتغطية هذه المنطقة. من ناحية أخرى، white_team #7 يظهر كفاءة في الهجمات المرتدة، لكنه بحاجة إلى دعم من زملائه لزيادة الفعالية." 
    """

    # يمكنك تغيير analysis_prompt ليسأل أسئلة مختلفة
    # analysis_prompt = "صف كيف تغير موقع الكرة خلال الفترة الزمنية المعطاة."
    # analysis_prompt = "هل يوجد لاعب معين من فريق 'red_team' يبدو أنه قضى معظم وقته في منطقة الهجوم؟"

    print(f"\nطلب التحليل المحدد:\n{analysis_prompt}")

    gemini_analysis = analyze_with_gemini(data_for_llm, analysis_prompt)

    print("\n--- تحليل Gemini للبيانات ---")
    print(gemini_analysis)
else:
    print("\nلم يتم جمع أي بيانات من الفيديو، لا يمكن إجراء تحليل Gemini.")

print("\n--- انتهى البرنامج ---")

In [ ]:
# --- التثبيت (تأكد من تشغيل هذا في بيئة Kaggle أو محليًا) ---
# !pip install Flask flask-ngrok supervision inference-sdk sports-analytic-tool google-generativeai opencv-python numpy werkzeug pyngrok # قد تحتاج لـ !pip install Flask==2.2.2 إذا واجهت مشاكل توافق
# !pip install --upgrade supervision # تأكد من أن لديك إصدار حديث من supervision

import os
import json
import threading
import uuid
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template_string, jsonify, send_from_directory, abort
from pyngrok import ngrok

# --- استيرادات الرؤية الحاسوبية والتحليل ---
import numpy as np
import supervision as sv
from inference.models.utils import get_roboflow_model
from sports.configs.soccer import SoccerPitchConfiguration
from sports.common.view import ViewTransformer
import cv2
import google.generativeai as genai
try:
    from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
except ImportError:
    print("Warning: Could not import sports.annotators.soccer. Pitch drawing might fail.")
    def draw_pitch(config): return np.zeros((200, 300, 3), dtype=np.uint8)
    def draw_points_on_pitch(config, xy, face_color, edge_color, radius, thickness, pitch): return pitch

# --- إعدادات وتكوينات أساسية ---
UPLOAD_FOLDER = '/kaggle/working/uploads'
OUTPUT_FOLDER = '/kaggle/working/outputs'
ALLOWED_EXTENSIONS = {'mp4', 'avi', 'mov'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER
app.secret_key = 'a_very_secret_random_key_too'
ngrok_tunnel = None

os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)



ROBOFLOW_API_KEY = "your_api_key"  # <--- استبدل بمفتاح Roboflow الخاص بك
GEMINI_API_KEY_HARDCODED = "your_api_key" # <--- المفتاح الذي تستخدمه للاختبار

# تحميل نماذج Roboflow
try:
    print("Loading Roboflow models...")
    if not ROBOFLOW_API_KEY or ROBOFLOW_API_KEY == "YOUR_ROBOFLOW_API_KEY_HERE": # تحقق إضافي
        print("Error: Roboflow API Key is not set. Please replace 'YOUR_ROBOFLOW_API_KEY_HERE'.")
        exit()
    model = get_roboflow_model(model_id="cat-nifqa/1", api_key=ROBOFLOW_API_KEY)
    FIELD_DETECTION_MODEL = get_roboflow_model(model_id="football-field-detection-f07vi/14", api_key=ROBOFLOW_API_KEY)
    class_names = model.class_names
    print("Roboflow Models loaded successfully.")
except Exception as e:
    print(f"Error loading Roboflow models: {e}")
    exit()


GEMINI_API_KEY = None
try:
    GEMINI_API_KEY = GEMINI_API_KEY_HARDCODED
    if not GEMINI_API_KEY:
        print("Error: Gemini API Key is empty.")
        exit()
    else:
        genai.configure(api_key=GEMINI_API_KEY)
        print("Gemini API Key configured (using hardcoded key - NOT RECOMMENDED FOR PRODUCTION).")
except Exception as e:
    print(f"Error configuring Gemini API key: {e}")
    exit()


BALL_CLASS_ID = 0
CONFIG = SoccerPitchConfiguration()




def classify_tactical_zone(x: float, y: float, is_home_team: bool, pitch_config: SoccerPitchConfiguration) -> str:
    pitch_width = 12000.0 
    pitch_height = 8000.0

    
    first_third_line_x = pitch_width / 3
    second_third_line_x = pitch_width * (2 / 3)

    
    flank_third_line_y1 = pitch_height / 3
    flank_third_line_y2 = pitch_height * (2 / 3)

    zone_base = "Unknown Zone"

    
    if is_home_team:
        if x > second_third_line_x: zone_base = "Attacking Third"
        elif x < first_third_line_x: zone_base = "Defensive Third"
        else: zone_base = "Midfield Third"
    else: # الفريق الضيف يهاجم نحو x الأصغر
        if x < first_third_line_x: zone_base = "Attacking Third"
        elif x > second_third_line_x: zone_base = "Defensive Third"
        else: zone_base = "Midfield Third"

    # تحديد المنطقة العرضية (جناح/مركز)
    zone_modifier = ""
    if zone_base != "Unknown Zone":
        if y < flank_third_line_y1: zone_modifier = " (Top Flank)" # Y صغير = أعلى
        elif y > flank_third_line_y2: zone_modifier = " (Bot Flank)" # Y كبير = أسفل
        else: zone_modifier = " (Center)"

    return zone_base + zone_modifier


def analyze_with_gemini(data_list, analysis_request):
    print("--- Entering analyze_with_gemini ---") 
    if not data_list:
        print("--- No data found in analyze_with_gemini ---")
        return "لا توجد بيانات لتحليلها."

    
    if not GEMINI_API_KEY:
         print("--- analyze_with_gemini: GEMINI_API_KEY is unexpectedly None or empty! ---")
         return "خطأ داخلي: مفتاح Gemini API غير موجود."

    data_json = None 
    try:
        print("--- Attempting json.dumps ---")
        data_json = json.dumps(data_list, indent=2, ensure_ascii=False)
       
        # print(f"--- json.dumps successful (first 200 chars): {data_json[:200]} ... ---")
        print("--- json.dumps successful ---")
    except Exception as e:
        print(f"--- json.dumps failed: {e} ---") 
        return f"خطأ في تحويل البيانات إلى JSON: {e}"

    
    if data_json is None:
         print("--- ERROR: data_json is None after first try block! ---")
         return "خطأ داخلي: فشل إنشاء JSON بشكل غير متوقع."

    try:
        print(f"--- Attempting Gemini call. data_json type: {type(data_json)}, approx length: {len(data_json)} bytes ---") 
       
        # gemini_model = genai.GenerativeModel('gemini-1.5-pro-latest')
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')

        prompt = f"""
        تحليل بيانات مباراة كرة قدم:

        البيانات التالية تم استخراجها من مقطع فيديو لمباراة كرة قدم. كل عنصر في القائمة يمثل بيانات إطار معين تم التقاطه كل ثانية (تقريبًا).
        كل إطار يحتوي على قائمة بالكائنات المكتشفة (لاعبين، الكرة) مع معرفاتها، نوعها (الفريق أو الكرة)، موقعها التقريبي في الملعب (الإحداثيات المحولة)، والمنطقة التكتيكية التي يتواجد فيها اللاعب.

        البيانات:
        ```json
        {data_json}
        ```

        المطلوب:
        {analysis_request}

        يرجى تقديم التحليل باللغة العربية وبتنسيق واضح ومقسم.
        """
        # print(f"--- Prompt created (first 300 chars): {prompt[:300]}... ---") 
        print("--- Prompt created, calling generate_content ---")
        
        request_options = {"timeout": 300}
        response = gemini_model.generate_content(prompt, request_options=request_options)

        # response = gemini_model.generate_content(prompt)
        print("--- Gemini call successful (received response object) ---")

        
        if response.parts:
             print("--- Returning Gemini response text ---")
             
             return response.text.strip()
        else:
             
             print("--- Gemini response was empty or blocked ---")
             
             try:
                 print(f"Finish Reason: {response.candidates[0].finish_reason}")
                 print(f"Safety Ratings: {response.candidates[0].safety_ratings}")
             except (IndexError, AttributeError):
                 print("Could not retrieve candidate details from response.")
             print(f"Prompt Feedback: {response.prompt_feedback}")
             return "لم يتم استلام تحليل من Gemini (قد يكون الرد محظورًا بسبب سياسات الأمان أو فارغًا)."

    except Exception as e:
        
        print(f"--- Exception during Gemini call or prompt creation: {e} ---") 
        
        error_str = str(e).lower()
        if "api key not valid" in error_str:
            return "خطأ: مفتاح Gemini API المستخدم غير صالح. يرجى التحقق منه."
        elif "quota" in error_str:
            return "خطأ: تجاوز الحصة المخصصة لواجهة برمجة التطبيقات. حاول مرة أخرى لاحقًا أو تحقق من حدود حسابك."
        elif "resource has been exhausted" in error_str or "token" in error_str:
             return "خطأ: حجم البيانات المرسلة كبير جدًا وقد يتجاوز حدود النموذج. حاول معالجة مقطع فيديو أقصر."
        elif "deadline exceeded" in error_str:
             return "خطأ: استغرق الطلب وقتًا طويلاً جدًا للرد (Deadline Exceeded). حاول مرة أخرى لاحقًا أو بمقطع فيديو أقصر."
        
        elif "userlocationinvalid" in error_str:
             return "خطأ: يبدو أن واجهة برمجة تطبيقات Gemini غير متاحة في موقعك الجغرافي الحالي."
        elif "500 internal server error" in error_str:
             return "خطأ: حدث خطأ داخلي في خادم Gemini (500). حاول مرة أخرى لاحقًا."
        
        return f"خطأ غير متوقع أثناء التفاعل مع Gemini API: {e}"



tasks = {} # task_id: {'status': 'processing'/'completed'/'error', 'input': 'path', 'output_filename': 'name', 'analysis': 'text', 'error_message': 'msg'}


def process_video_and_analyze(task_id, input_video_path, output_video_path):
    global tasks 

    
    local_data_for_llm = []
    local_persistent_ids = {}
    local_current_persistent_id = 1
    local_home_team = None
    local_away_team = None
    
    known_home_team = "white_team"
    known_away_team = "black_team"

    print(f"--- Task {task_id}: Initializing processing ---")
    try:
        
        tracker = sv.ByteTrack(
            # يمكنك تجربة هذه القيم إذا كان التتبع غير دقيق
            # track_activation_threshold=0.25,
            # lost_track_buffer=90, # 3 ثوانٍ تقريبًا بمعدل 30 إطارًا في الثانية
            # minimum_matching_threshold=0.8,
            frame_rate=25 # اضبط هذا لمعدل إطارات الفيديو الفعلي إذا كان معروفًا
        )
        
        trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=60)
        ellipse_annotator = sv.EllipseAnnotator(thickness=2)
        
        # label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1, text_padding=2)
        # triangle_annotator = sv.TriangleAnnotator(color=sv.Color.YELLOW, base=10, height=10)

        
        def local_callback(frame: np.ndarray, frame_id: int) -> np.ndarray:
            
            nonlocal local_home_team, local_away_team, local_current_persistent_id, local_data_for_llm, local_persistent_ids

            try: 
                # تنفيذ الاكتشاف
                results = model.infer(frame)[0]
                detections = sv.Detections.from_inference(results)

                
                ball_mask = detections.class_id == BALL_CLASS_ID
                ball_detections = detections[ball_mask]
                other_detections = detections[~ball_mask]

                if len(ball_detections) > 0:
                    ball_detections = ball_detections[ball_detections.confidence >= 0.1]
                   
                    # ball_detections = sv.Detections(xyxy=sv.pad_boxes(ball_detections.xyxy, 10), ...)
                if len(other_detections) > 0:
                    
                    other_detections = other_detections[other_detections.confidence >= 0.6].with_nms(threshold=0.5, class_agnostic=True)


                
                combined = sv.Detections.merge([ball_detections, other_detections])
                tracked_detections = tracker.update_with_detections(combined)

                
                current_tracker_ids = set(tracked_detections.tracker_id)
                for tracker_id in current_tracker_ids:
                    if tracker_id not in local_persistent_ids:
                        local_persistent_ids[tracker_id] = local_current_persistent_id
                        local_current_persistent_id += 1

                
                transformer = None
                pitch_anchors = np.empty((0, 2)) 
                try:
                    field_result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
                    key_points = sv.KeyPoints.from_inference(field_result)
                    
                    if key_points.confidence is not None and len(key_points.confidence) > 0 and key_points.xy is not None and len(key_points.xy) > 0:
                        confidence_mask = key_points.confidence[0] > 0.5
                        filtered_xy = key_points.xy[0][confidence_mask]
                        if len(filtered_xy) >= 4:
                            
                            pitch_points = np.array(CONFIG.vertices)[confidence_mask]
                            transformer = ViewTransformer(source=filtered_xy, target=pitch_points)

                            
                            if len(tracked_detections) > 0:
                                frame_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
                                pitch_anchors = transformer.transform_points(points=frame_anchors)
                        # else:
                            # print(f"Frame {frame_id}: Not enough high-confidence keypoints ({len(filtered_xy)})")
                    # else:
                        # print(f"Frame {frame_id}: No keypoints detected or invalid structure.")

                except Exception as e_transform:
                    
                    # print(f"Frame {frame_id}: Transformation setup/application failed: {str(e_transform)}")
                    if len(tracked_detections) > 0:
                        pitch_anchors = tracked_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER) 

                
                current_frame_data = {"frame_id": frame_id, "timestamp": round(frame_id / 25.0, 2), "objects": []} 

                
                detected_teams = set()
                for i in range(len(tracked_detections)):
                    class_id = tracked_detections.class_id[i]
                    if 0 <= class_id < len(class_names): 
                        cls_name = class_names[class_id]
                        
                        if "team" in cls_name and class_id != BALL_CLASS_ID:
                            detected_teams.add(cls_name)

                if not local_home_team and known_home_team in detected_teams: local_home_team = known_home_team
                if not local_away_team and known_away_team in detected_teams: local_away_team = known_away_team

                
                tracker_id_to_zone = {}
                for i in range(len(tracked_detections)):
                    tracker_id = tracked_detections.tracker_id[i]
                    class_id = tracked_detections.class_id[i]

                    if 0 <= class_id < len(class_names): 
                        class_name = class_names[class_id]
                        
                        position = pitch_anchors[i].tolist() if i < len(pitch_anchors) and pitch_anchors[i] is not None else None 

                        zone = "Unknown"
                        
                        if position is not None and "team" in class_name:
                            is_home = (class_name == local_home_team) if local_home_team else False
                            x, y = position 
                            zone = classify_tactical_zone(x, y, is_home, pitch_config=CONFIG)
                        tracker_id_to_zone[tracker_id] = zone

                        
                        persistent_id_val = local_persistent_ids.get(tracker_id, -1) 

                        obj = {
                            "persistent_id": int(persistent_id_val),
                            "tracker_id": int(tracker_id),
                            "class": class_name,
                            "confidence": float(tracked_detections.confidence[i]),
                            "position": position, 
                            "zone": zone
                        }
                        current_frame_data["objects"].append(obj)
                    # else:
                       

                
                if frame_id % 25 == 0: 
                    local_data_for_llm.append(current_frame_data)

                
                annotated_frame = frame.copy()
                
                # ...

                
                annotated_frame = ellipse_annotator.annotate(annotated_frame, tracked_detections)
                annotated_frame = trace_annotator.annotate(annotated_frame, tracked_detections)
                
                # labels = [f"ID:{tid} Z:{tracker_id_to_zone.get(tid, '?')}" for tid in tracked_detections.tracker_id]
                # annotated_frame = label_annotator.annotate(annotated_frame, tracked_detections, labels=labels)

                return annotated_frame

            except Exception as e_frame:
                print(f"--- Task {task_id}: Error processing frame {frame_id}: {e_frame} ---")
                
                return frame
            #

        # --- بدء معالجة الفيديو ---
        print(f"--- Task {task_id}: Starting video processing (sv.process_video) ---")
        sv.process_video(
            source_path=input_video_path,
            target_path=output_video_path,
            callback=local_callback
        )
        print(f"--- Task {task_id}: Video processing completed. Output: {output_video_path} ---")
        print(f"--- Task {task_id}: Collected {len(local_data_for_llm)} data points for LLM analysis. ---")

        
        if not local_data_for_llm:
             print(f"--- Task {task_id}: No data collected for LLM, skipping analysis. ---")
             analysis_result = "لم يتم جمع بيانات كافية من الفيديو لإجراء التحليل."
        else:
            print(f"--- Task {task_id}: Starting Gemini analysis ---")
            
            analysis_prompt = """
            الهدف:
تحليل بيانات الرؤية الحاسوبية لمباراة كرة قدم لاستخراج نقاط القوة ، نقاط الضعف ، التكتيكات الفعالة ، والمجالات التي تحتاج تحسينًا ، مع تقديم توصيات عملية للمدرب.

التعليمات :
تحليل حركة اللاعبين والكرة:
تتبع مسارات اللاعبين والكرة عبر الإطارات لتحديد:
نقاط القوة : لاعبون ذوي حركة سريعة أو دقة في التمركز (مثال: white_team #7 في الهجوم).
نقاط الضعف : لاعبون بطيئون أو خارج التغطية الدفاعية (مثال: black_team #24 في المناطق الدفاعية).
اكتشاف أنماط التسارع/التباطؤ التي تشير إلى إجهاد أو انضباط تكتيكي.
تحليل السيطرة على المناطق:
حدد المناطق التي يُسيطر عليها كل فريق (مثل ثلث الهجوم أو الدفاع).
ابحث عن ثغرات :
هل يوجد تركز مفرط للاعبي الفريق الأبيض في الوسط، مما يعرض الدفاع للخطر؟
هل الفريق الأسود يستغل الأطراف بشكل كافٍ؟
اكتشاف الأحداث الحرجة:
الاستحواذ : لاعبون قريبون من الكرة مع Conf ≥ 0.85 (مثال: black_team #19 في منتصف الملعب).
التمريرات الناجحة/الفاشلة : تمريرات طويلة أو قصيرة تشير إلى تنظيم هجومي أو فوضى.
التسديدات : كرات نحو المرمى مع ملاحظة مدى قرب المدافع (مثال: تسديدة white_team #9 في الإطار 600).
التدخلات الدفاعية : اعتراضات أو تدخلات فاشلة تكشف عن ضعف في التغطية.
تقييم الاستراتيجيات:
نقاط القوة التكتيكية :
هل يستخدم الفريق الأبيض تشكيلًا دفاعيًا فعالًا (مثل 4-4-2)؟
هل الفريق الأسود يشن هجمات مرتدة سريعة؟
نقاط الضعف التكتيكية :
هل هناك فجوات بين خطوط الفريق (مثل مسافة كبيرة بين الدفاع والوسط)؟
هل الحراس أو المدافعون غير قادرين على التعامل مع الكرات العرضية؟
تقييم الأداء الفردي:
لاعبون بـ تفاعل عالٍ : مشاركة في أغلب الأحداث (مثال: black_team #19 في الوسط).
لاعبون بـ أداء ضعيف : أخطاء متكررة أو مواقع غير مناسبة (مثال: red_team #23 مع Conf < 0.80).
اقترح مراجعة الفيديو للتأكد من الأحداث الحرجة (مثل التسديدات أو التدخلات).
متطلبات الإخراج المُعدلة:
ملخص تنفيذي للمدرب:
نقاط القوة الرئيسية لكل فريق (مثال: "سيطرة black_team على الوسط").
نقاط الضعف الحرجة (مثال: "ضعف تغطية white_team للأطراف").
توصيات فورية (مثال: "تدريب black_team #24 على السرعة في المناطق الدفاعية").
وصف بصري تكتيكي:
مخططات نصية لتمركز اللاعبين في أحداث حرجة (مثال: "في الإطار 600، white_team #7 في موقع تسديد مثالي").

مثال تحليل للمدرب:

"في الإطار 600، يظهر black_team #19 سيطرة واضحة على الوسط، لكنه يترك مساحة كبيرة في الجناح الأيمن. يُوصى بتحريك black_team #12 لتغطية هذه المنطقة. من ناحية أخرى، white_team #7 يظهر كفاءة في الهجمات المرتدة، لكنه بحاجة إلى دعم من زملائه لزيادة الفعالية." 
            """
            analysis_result = analyze_with_gemini(local_data_for_llm, analysis_prompt)

        print(f"--- Task {task_id}: Gemini analysis completed. ---")

        
        tasks[task_id]['status'] = 'completed'
        tasks[task_id]['analysis'] = analysis_result
        

    except Exception as e:
        
        print(f"--- Task {task_id}: Error during processing: {e} ---")
        import traceback
        traceback.print_exc() 
        tasks[task_id]['status'] = 'error'
        
        error_msg = str(e) if e else "Unknown error during processing."
        tasks[task_id]['error_message'] = f"فشل في المعالجة: {error_msg}"



def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS



HTML_TEMPLATE_CHATGPT_STYLE = """
<!doctype html>
<html lang="ar" dir="rtl">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>تحليل تكتيكي متقدم</title>
    <!-- Google Fonts (e.g., Sora) -->
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Sora:wght@300;400;600&display=swap" rel="stylesheet">
    <!-- Font Awesome for Icons (Optional) -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css" integrity="sha512-DTOQO9RWCH3ppGqcWaEA1BIZOC6xxalwEsw9c2QQeAIftl+Vegovlnee1c9QX4TctnWMn13TZye+giMm8e2LwA==" crossorigin="anonymous" referrerpolicy="no-referrer" />

    <style>
        /* CSS Variables for Theming (Dark Theme) */
        :root {
            --bg-primary: #202123;       /* Main background */
            --bg-secondary: #343541;    /* Card/Input background */
            --bg-tertiary: #444654;     /* Hover/Accent background */
            --text-primary: #ffffff;    /* Main text */
            --text-secondary: #b0b0b0; /* Subdued text */
            --border-color: #565869;    /* Borders */
            --accent-color: #10a37f;    /* Accent color (buttons, links) */
            --error-color: #ef4444;     /* Error messages */
            --font-family: 'Sora', sans-serif; /* Font */
            --border-radius: 8px;
        }

        /* Basic Reset & Body Styling */
        * { box-sizing: border-box; margin: 0; padding: 0; }
        html { scroll-behavior: smooth; }
        body {
            font-family: var(--font-family);
            background-color: var(--bg-primary);
            color: var(--text-primary);
            line-height: 1.6;
            display: flex;
            flex-direction: column;
            min-height: 100vh; /* Ensure footer stays at bottom */
            padding: 1rem;
        }

        /* Main Container */
        .chat-container {
            max-width: 950px;
            margin: 2rem auto;
            width: 100%;
            background-color: var(--bg-secondary);
            border-radius: var(--border-radius);
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.2);
            overflow: hidden; /* Contain children */
            flex-grow: 1; /* Allow container to grow */
            display: flex;
            flex-direction: column;
        }

        /* Header */
        header {
            padding: 1.5rem;
            text-align: center;
            border-bottom: 1px solid var(--border-color);
        }
        header h1 { font-size: 1.8rem; font-weight: 600; color: var(--accent-color); }
        header p { color: var(--text-secondary); font-size: 0.9rem; }

        /* Main Content Area (Scrollable) */
        .main-content {
             flex-grow: 1;
             overflow-y: auto; /* Make content scrollable */
             padding: 1.5rem;
        }

        /* Input/Upload Section */
        .input-section {
            padding: 1.5rem;
            background-color: rgba(0,0,0,0.1); /* Slightly darker */
            border-top: 1px solid var(--border-color);
        }
        .input-section h2 {
            font-size: 1.1rem;
            margin-bottom: 1rem;
            font-weight: 400;
            color: var(--text-secondary);
        }
        .input-group { display: flex; gap: 0.5rem; }
        .file-input-wrapper {
            position: relative;
            overflow: hidden;
            display: inline-block;
            cursor: pointer;
            padding: 0.6rem 1rem;
            background-color: var(--bg-tertiary);
            border: 1px solid var(--border-color);
            border-radius: var(--border-radius);
            color: var(--text-primary);
            transition: background-color 0.2s ease;
            flex-grow: 1; /* Take remaining space */
             text-align: right; /* Align text right */
        }
        .file-input-wrapper:hover { background-color: #565869; }
        .file-input-wrapper input[type=file] {
            position: absolute;
            left: 0;
            top: 0;
            opacity: 0;
            cursor: pointer;
            width: 100%;
            height: 100%;
        }
        .file-input-label {
            font-size: 0.9rem;
            color: var(--text-secondary);
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
            display: block; /* Ensure it takes width */
        }
         .file-input-wrapper i { margin-left: 0.5rem; } /* Icon spacing (Arabic) */

        .btn-submit {
            background-color: var(--accent-color);
            color: var(--text-primary);
            border: none;
            padding: 0.6rem 1.2rem;
            border-radius: var(--border-radius);
            cursor: pointer;
            transition: background-color 0.2s ease;
            font-weight: 600;
            font-family: var(--font-family);
            white-space: nowrap;
        }
        .btn-submit:hover { background-color: #0d8a6a; }
        .btn-submit:disabled { background-color: var(--bg-tertiary); color: var(--text-secondary); cursor: not-allowed; }
         .btn-submit i { margin-left: 0.5rem; } /* Icon spacing */

        /* Results Section */
        .results-section {
            margin-top: 1.5rem;
            padding-top: 1.5rem;
            border-top: 1px solid var(--border-color);
            display: none; /* Hidden initially */
        }
        .results-section h2 { font-size: 1.2rem; margin-bottom: 1rem; text-align: center; color: var(--text-secondary); }

        /* Loading Indicator */
        .loading-indicator { text-align: center; padding: 2rem 0; }
        .spinner {
            border: 4px solid rgba(255, 255, 255, 0.2);
            border-left-color: var(--accent-color);
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto 1rem auto;
        }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        .loading-indicator p { color: var(--text-secondary); font-size: 0.9rem; }

        /* Result Content Area */
        .result-content { display: none; } /* Hidden until results ready */
        .result-card {
            background-color: var(--bg-tertiary);
            border: 1px solid var(--border-color);
            border-radius: var(--border-radius);
            padding: 1.5rem;
            margin-bottom: 1.5rem;
        }
        .result-card h4 {
            font-size: 1rem;
            margin-bottom: 1rem;
            color: var(--accent-color);
            border-bottom: 1px solid var(--border-color);
            padding-bottom: 0.5rem;
            font-weight: 600;
        }

        /* Analysis Text Styling */
        .analysis-result {
            white-space: pre-wrap;
            font-size: 0.95rem;
            max-height: 400px; /* Limit height and allow scrolling */
            overflow-y: auto;
            background-color: rgba(0,0,0,0.1); /* Slightly different background */
            padding: 1rem;
            border-radius: calc(var(--border-radius) - 4px);
            color: var(--text-primary);
        }
         /* Scrollbar styling (optional, webkit browsers) */
        .analysis-result::-webkit-scrollbar { width: 8px; }
        .analysis-result::-webkit-scrollbar-track { background: var(--bg-secondary); border-radius: 4px; }
        .analysis-result::-webkit-scrollbar-thumb { background-color: var(--border-color); border-radius: 4px; border: 2px solid var(--bg-secondary); }
        .analysis-result::-webkit-scrollbar-thumb:hover { background-color: #6a6c7b; }

        /* Video Styling */
        .video-result video {
            display: block;
            max-width: 100%;
            height: auto;
            border-radius: calc(var(--border-radius) - 4px);
            background-color: #000; /* Black background for video player */
            margin: 0 auto; /* Center video */
        }
        .video-download-link {
             display: none; /* Hidden initially */
             text-align: center;
             margin-top: 1rem;
        }
         .video-download-link a {
             color: var(--accent-color);
             text-decoration: none;
             font-size: 0.85rem;
             padding: 0.3rem 0.8rem;
             border: 1px solid var(--accent-color);
             border-radius: var(--border-radius);
             transition: background-color 0.2s ease, color 0.2s ease;
         }
         .video-download-link a:hover {
             background-color: var(--accent-color);
             color: var(--bg-secondary);
         }

        /* Error Message Styling */
        .error-result {
            color: var(--error-color);
            background-color: rgba(239, 68, 68, 0.1);
            border: 1px solid var(--error-color);
            padding: 1rem;
            border-radius: var(--border-radius);
            display: none; /* Hidden initially */
            margin-top: 1rem;
            text-align: center;
            font-size: 0.9rem;
        }

        /* Footer */
        footer {
             text-align: center;
             padding: 1rem;
             font-size: 0.8rem;
             color: var(--text-secondary);
             margin-top: auto; /* Push footer to bottom */
        }

         /* Responsive Adjustments */
         @media (max-width: 768px) {
             body { padding: 0.5rem; }
             .chat-container { margin: 0.5rem auto; }
             header { padding: 1rem; }
             header h1 { font-size: 1.5rem; }
             .main-content, .input-section { padding: 1rem; }
             .input-group { flex-direction: column; }
             .btn-submit { width: 100%; margin-top: 0.5rem; }
             .result-card { padding: 1rem; }
         }

    </style>
</head>
<body>
    <div class="chat-container">
        <header>
            <h1><i class="fas fa-robot"></i> تحليل تكتيكي متقدم <i class="fas fa-futbol"></i></h1>
            <p>ارفع فيديو المباراة لتحليل تكتيكي بواسطة الذكاء الاصطناعي</p>
        </header>

        <main class="main-content">
            <!-- النتائج ستظهر هنا -->
            <section class="results-section" id="resultsSection">
                 <!-- Loading Indicator -->
                 <div class="loading-indicator" id="loadingIndicator" style="display: none;">
                    <div class="spinner"></div>
                    <p>جاري التحليل العميق... قد يستغرق هذا بعض الوقت.</p>
                 </div>

                 <!-- Result Content (Video & Analysis) -->
                 <div class="result-content" id="resultContent">
                     <div class="result-card">
                         <h4><i class="fas fa-video"></i> الفيديو المعالج</h4>
                         <div class="video-result" id="videoResult">
                             <!-- Video player/placeholder will be inserted here by JS -->
                         </div>
                         <div class="video-download-link" id="videoDownloadLink">
                             <a href="#" target="_blank" download><i class="fas fa-download"></i> تحميل الفيديو</a>
                         </div>
                     </div>
                     <div class="result-card">
                         <h4><i class="fas fa-brain"></i> رؤى وتحليلات Gemini</h4>
                         <div class="analysis-result" id="analysisResult">
                             <!-- Analysis text will be inserted here by JS -->
                         </div>
                     </div>
                 </div>

                 <!-- Error Message Display -->
                 <div class="error-result" id="errorResult"></div>
            </section>
        </main>

        <!-- Input Area at the bottom -->
        <section class="input-section">
            <form method=post enctype=multipart/form-data id="uploadForm">
                 <div class="input-group">
                     <label class="file-input-wrapper">
                         <input type="file" name="file" id="videoFile" accept=".mp4,.avi,.mov" required>
                         <span class="file-input-label" id="fileNameLabel"><i class="fas fa-file-video"></i> اختر ملف الفيديو...</span>
                     </label>
                     <button type="submit" class="btn-submit" id="submitButton">
                         <i class="fas fa-paper-plane"></i> إرسال للتحليل
                     </button>
                 </div>
            </form>
        </section>

    </div> <!-- End chat-container -->

     <footer>
        تطبيق تجريبي - قد تكون النتائج غير دقيقة.
    </footer>

    <script>
        const uploadForm = document.getElementById('uploadForm');
        const submitButton = document.getElementById('submitButton');
        const videoFile = document.getElementById('videoFile');
        const fileNameLabel = document.getElementById('fileNameLabel');
        const resultsSection = document.getElementById('resultsSection');
        const loadingIndicator = document.getElementById('loadingIndicator');
        const resultContent = document.getElementById('resultContent');
        const errorResult = document.getElementById('errorResult');
        const analysisResult = document.getElementById('analysisResult');
        const videoResult = document.getElementById('videoResult');
        const videoDownloadLink = document.getElementById('videoDownloadLink').querySelector('a'); // Get the link inside

        // Update label when a file is selected
        videoFile.addEventListener('change', function() {
            if (videoFile.files.length > 0) {
                fileNameLabel.textContent = videoFile.files[0].name;
            } else {
                fileNameLabel.innerHTML = '<i class="fas fa-file-video"></i> اختر ملف الفيديو...';
            }
        });

        uploadForm.addEventListener('submit', function(event) {
            event.preventDefault();

            if (!videoFile.files || videoFile.files.length === 0) {
                showError('الرجاء اختيار ملف فيديو أولاً.');
                return;
            }

            submitButton.disabled = true;
            submitButton.innerHTML = '<i class="fas fa-spinner fa-spin"></i> جاري الرفع...';
            resultsSection.style.display = 'block'; // Show results area
            loadingIndicator.style.display = 'block'; // Show spinner
            resultContent.style.display = 'none'; // Hide old results
            errorResult.style.display = 'none'; // Hide old errors
            analysisResult.textContent = ''; // Clear previous analysis
            videoResult.innerHTML = ''; // Clear previous video
            videoDownloadLink.parentElement.style.display = 'none'; // Hide download link div

            const formData = new FormData(uploadForm);

            fetch('/', { method: 'POST', body: formData })
                .then(response => {
                    if (!response.ok) {
                        // Try to get error message from server if possible
                         return response.json().then(errData => {
                             throw new Error(errData.error || `خطأ في الخادم: ${response.status}`);
                         }).catch(() => {
                            // If no JSON error body, throw generic error
                            throw new Error(`خطأ في الخادم: ${response.status}`);
                         });
                    }
                    return response.json();
                })
                .then(data => {
                    submitButton.innerHTML = '<i class="fas fa-cogs"></i> قيد المعالجة...'; // Update button text
                    if (data.task_id) {
                        checkTaskStatus(data.task_id);
                    } else {
                         throw new Error(data.error || 'استجابة غير متوقعة من الخادم عند بدء المهمة.');
                    }
                })
                .catch(error => {
                    console.error('Error submitting form:', error);
                    showError(`فشل بدء التحليل: ${error.message}`);
                    resetSubmitButton();
                });
        });

        function checkTaskStatus(taskId) {
            console.log(`Checking status for task: ${taskId}`);
            fetch(`/status/${taskId}`)
                .then(response => {
                     if (!response.ok) { throw new Error(`خطأ ${response.status} عند التحقق.`); }
                    return response.json();
                })
                .then(data => {
                    if (data.status === 'processing') {
                        // Update loading text maybe?
                         loadingIndicator.querySelector('p').textContent = 'ما زال التحليل العميق مستمراً...';
                        setTimeout(() => checkTaskStatus(taskId), 7000); // Check every 7 seconds
                    } else if (data.status === 'completed') {
                        loadingIndicator.style.display = 'none';
                        analysisResult.textContent = data.analysis || 'لم يتم تقديم تحليل.'; // Display analysis

                        if (data.output_filename) {
                            const videoUrl = `/outputs/${data.output_filename}`;
                            const videoElement = document.createElement('video');
                            videoElement.src = videoUrl;
                            videoElement.controls = true;
                            videoElement.preload = 'metadata';
                            videoElement.style.maxWidth = '100%'; // Ensure video fits container
                            videoResult.innerHTML = ''; // Clear placeholder/old video
                            videoResult.appendChild(videoElement);

                            videoDownloadLink.href = videoUrl;
                            videoDownloadLink.download = data.output_filename;
                            videoDownloadLink.parentElement.style.display = 'block'; // Show download link div
                        } else {
                            videoResult.innerHTML = '<p class="text-warning">لم يتم العثور على الفيديو الناتج.</p>';
                             videoDownloadLink.parentElement.style.display = 'none';
                        }
                        resultContent.style.display = 'block'; // Show results card
                        errorResult.style.display = 'none';
                        resetSubmitButton();
                        // Scroll to results smoothly
                        resultsSection.scrollIntoView({ behavior: 'smooth' });
                    } else if (data.status === 'error') {
                         throw new Error(data.error_message || 'فشل التحليل لسبب غير معروف.');
                    } else {
                        throw new Error(`حالة مهمة غير معروفة: ${data.status}`);
                    }
                })
                .catch(error => {
                    console.error('Error checking status:', error);
                    showError(`فشل التحقق من حالة المهمة: ${error.message}`);
                    resetSubmitButton();
                });
        }

         function showError(message) {
             loadingIndicator.style.display = 'none';
             resultContent.style.display = 'none'; // Hide results content on error
             errorResult.textContent = message;
             errorResult.style.display = 'block';
             resultsSection.style.display = 'block'; // Ensure the section containing the error is visible
             resultsSection.scrollIntoView({ behavior: 'smooth' }); // Scroll to error
         }

         function resetSubmitButton() {
             submitButton.disabled = false;
             submitButton.innerHTML = '<i class="fas fa-paper-plane"></i> إرسال للتحليل';
         }
    </script>
</body>
</html>
"""



@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return jsonify({'error': 'لم يتم إرسال أي ملف.'}), 400
        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'لم يتم تحديد أي ملف.'}), 400

        if file and allowed_file(file.filename):
            try:
                original_filename = secure_filename(file.filename)
                unique_id = str(uuid.uuid4())
                input_filename = f"{unique_id}_{original_filename}"
                input_path = os.path.join(app.config['UPLOAD_FOLDER'], input_filename)

                output_filename = f"output_{input_filename}"
                output_path = os.path.join(app.config['OUTPUT_FOLDER'], output_filename)

                file.save(input_path)

                task_id = unique_id
                tasks[task_id] = {
                    'status': 'processing',
                    'input': input_path,
                    'output_filename': output_filename,
                    'analysis': None,
                    'error_message': None
                }

                print(f"--- Task {task_id}: Created. Input: {input_path}, Output: {output_path} ---")
                thread = threading.Thread(target=process_video_and_analyze, args=(task_id, input_path, output_path))
                thread.daemon = True
                thread.start()
                return jsonify({'task_id': task_id})

            except Exception as e:
                 print(f"Error handling file upload or starting task: {e}")
                 
                 return jsonify({'error': f'فشل داخلي في الخادم عند بدء المعالجة: {str(e)}'}), 500
        else:
             return jsonify({'error': 'نوع الملف غير مسموح به. استخدم mp4, avi, mov.'}), 400

    
    return render_template_string(HTML_TEMPLATE_CHATGPT_STYLE) 


@app.route('/status/<task_id>')
def task_status(task_id):
    task = tasks.get(task_id)
    if not task:
        return jsonify({'status': 'error', 'error_message': 'Task ID not found'}), 404

    response_data = {'status': task['status']}
    if task['status'] == 'completed':
        response_data['analysis'] = task.get('analysis', 'Analysis result not found.')
        response_data['output_filename'] = task.get('output_filename')
    elif task['status'] == 'error':
       
        error_msg = task.get('error_message', 'Unknown processing error.')
        response_data['error_message'] = str(error_msg) if error_msg else 'Unknown processing error.'


    return jsonify(response_data)


@app.route('/outputs/<filename>')
def serve_output_file(filename):
    try:
        safe_path = os.path.join(app.config['OUTPUT_FOLDER'], filename)
        if not os.path.abspath(safe_path).startswith(os.path.abspath(app.config['OUTPUT_FOLDER'])):
             abort(403)
        return send_from_directory(app.config['OUTPUT_FOLDER'], filename, as_attachment=False)
    except FileNotFoundError:
         abort(404)
    except Exception as e:
        print(f"Error serving file {filename}: {e}")
        abort(500)


if __name__ == "__main__":
    print("Starting Flask app with ngrok...")
    try:
        ngrok_tunnel = ngrok.connect(5000, "http")
        public_url = ngrok_tunnel.public_url
        print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")
        print(f" * Public URL: {public_url}")
        print("Flask app running...")
        app.run(port=5000, debug=False, use_reloader=False)

    except ImportError:
        print("Error: pyngrok not installed. Cannot create public tunnel.")
        print("Install using: pip install pyngrok")
        print("Attempting to run Flask locally on port 5000...")
        app.run(port=5000, debug=True)
    except Exception as e:
         print(f"An error occurred with ngrok or Flask app startup: {e}")
         
         if "authentication failed" in str(e).lower():
             print("Hint: Check your ngrok authtoken if you have one configured.")
         elif "tcp" in str(e).lower() and "bind" in str(e).lower():
              print("Hint: Port 5000 might already be in use by another application.")
         print("Attempting to run Flask locally on port 5000...")
         app.run(port=5000, debug=True)

    finally:
       if ngrok_tunnel:
            try:
                print("Disconnecting ngrok tunnel...")
                ngrok.disconnect(ngrok_tunnel.public_url)
            except Exception as e:
                print(f"Error disconnecting ngrok tunnel {ngrok_tunnel.public_url}: {e}")
       print("Flask app stopped.")

Loading Roboflow models...
Roboflow Models loaded successfully.
Gemini API Key configured (using hardcoded key - NOT RECOMMENDED FOR PRODUCTION).
Starting Flask app with ngrok...
 * ngrok tunnel "https://9e93-130-211-250-206.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Public URL: https://9e93-130-211-250-206.ngrok-free.app
Flask app running...
 * Serving Flask app '__main__'
 * Debug mode: off
--- Task 6163bba7-88e2-41c4-935c-99b024f82666: Created. Input: /kaggle/working/uploads/6163bba7-88e2-41c4-935c-99b024f82666_2e57b9_0.mp4, Output: /kaggle/working/outputs/output_6163bba7-88e2-41c4-935c-99b024f82666_2e57b9_0.mp4 ---
--- Task 6163bba7-88e2-41c4-935c-99b024f82666: Initializing processing ---
--- Task 6163bba7-88e2-41c4-935c-99b024f82666: Starting video processing (sv.process_video) ---
--- Task 6163bba7-88e2-41c4-935c-99b024f82666: Video processing completed. Output: /kaggle/working/outputs/output_6163bba7-88e2-41c4-935c-99b024f82666_2e57b9_0.mp4 ---
--- Task 6163bba7-88e2-41c4-935

In [ ]:
import json
import numpy as np

def convert_np_types(obj):
    """Recursively converts NumPy types to native Python types."""
    if isinstance(obj, (np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert NumPy arrays to lists
    elif isinstance(obj, (dict, list)):
        return [convert_np_types(v) for v in obj] if isinstance(obj, list) else {k: convert_np_types(v) for k, v in obj.items()}
    else:
        return obj

with open('match_data.json2', 'w') as f:
    # Convert NumPy types recursively before dumping to JSON
    modified_data = convert_np_types(data_for_llm)
    json.dump(modified_data, f, indent=2)

In [ ]:
import json

with open('match_data.json2', 'r') as f:
    match_data = json.load(f)

for frame_info in match_data:
    print(f"Frame {frame_info['frame_id']} ({frame_info['timestamp']}s):")

    # طباعة الكائنات مع المنطقة مع تجاهل الحكام
    for obj in frame_info['objects']:
        # تخطي الطباعة إذا كان الكائن حكمًا (class 5)
        if obj['class'] == 'referee':
            continue
            
        print(f"- {obj['class']} #{obj['tracker_id']}  "
              f"(Conf: {obj['confidence']:.2f}) at {obj['position']} "
              f"[Zone: {obj.get('zone', 'Unknown')}]")

    # # طباعة النقاط المهمة للملعب
    # if 'keypoints' in frame_info and len(frame_info['keypoints']) > 0:
    #     print("  Keypoints:")
    #     for kp in frame_info['keypoints']:
    #         print(f"  - Position: ({kp['position'][0]:.1f}, {kp['position'][1]:.1f}) "
    #               f"Confidence: {kp['confidence']:.2f}")